# Optical Frequency Response Measurement
This notebook provides a step-by-step workflow for automating the measurement and analysis of optical frequency response in photonic experiments.
It leverages PyVISA to control laboratory instruments, load experiment configurations, acquire measurement data, and visualize results.

**Workflow Overview:**
1. Import required libraries and initialize instruments.
2. Load experiment configuration parameters.
3. Perform automated laser and sourcemeter sweeps.
4. Analyze and visualize measurement data.
5. Save results for further analysis.

---

In [ ]:
import pyvisa as visa
from instruments.agilent8163 import Agilent8163Multimeter
from instruments.keithley2400 import Keithley2400SourceMeter
from controllers.laser_sweep import *
from controllers.sourcemeter_sweep import measure_iv_curve, measure_liv_curve
from utils.load_config import load_config
from utils.plotter import plot_measurements
from utils.data_saver import save_raw_measurements

## VISA Resource Discovery
This section lists all available VISA resources to ensure the required instruments are connected and recognized by the system before proceeding with measurements.

In [ ]:
from utils.visa_utils import list_visa_resources

# List all VISA resources
list_visa_resources()

In [ ]:
config = load_config('configs/experiment_config.yaml')
laser_cfg = config["Agilent8163Multimeter"]                         # Extract laser-specific configuration

combined_name = f"{config['DUT']['type']}_{config['DUT']['name']}"  # Combined name for saving data

In [ ]:
rm = visa.ResourceManager()

# Generate the laser and keithley objects using the configuration
laser_obj = Agilent8163Multimeter(
    address=laser_cfg['address'],
    laser_slot=laser_cfg['laser_slot'],
    power_slot= laser_cfg['power_slot'],
    power_channel=laser_cfg['power_channel'],
    resource_manager=rm
)

## Laser Sweep Measurement and Analysis
This section performs the automated laser sweep, identifies the peak response, and visualizes the measurement results for analysis.

In [ ]:
laser_sweep_params = {
  "laser": laser_obj,
  "start_wl": laser_cfg["start_wavelength"],
  "stop_wl": laser_cfg["stop_wavelength"],
  "step": laser_cfg["step"],
  "delay": laser_cfg["delay"],
  "logger": None,
  "live_plot": True,
}

headers_laser_sweep, results_laser_sweep = perform_laser_sweep(**laser_sweep_params)

# Get the maximum power and its corresponding wavelength
wavelengths, powers = np.array(results_laser_sweep).T
max_idx = powers.argmax()
mzi_peak = powers[max_idx]
mzi_peak_wavelength = wavelengths[max_idx]
print(f"Peak power: {mzi_peak} at wavelength: {mzi_peak_wavelength} nm")
# Plot the laser sweep results
plot_path = plot_measurements(headers=headers_laser_sweep, 
           results= results_laser_sweep, 
           figure_name=combined_name + "_Laser_Sweep", 
           show=True,
           )

## Plotting and Saving Results
This section visualizes the laser sweep data and saves the raw measurement results to a CSV file for documentation and further analysis.

In [ ]:
# Plot the laser sweep results
plot_path = plot_measurements(headers=headers_laser_sweep, 
           results= results_laser_sweep, 
           figure_name=combined_name + "_Laser_Sweep", 
           show=True,
           )

# Save the laser sweep results to a CSV file
data_path = save_raw_measurements(headers=headers_laser_sweep,
                      data=results_laser_sweep,
                      filename=combined_name + "_Laser_Sweep",
)